### Data Wrangling: Join, Combine, Reshape Data

#### Combining Datasets: Concat and Append ( )

Some of the most interesting studies of data come from combining different data sources. These operations can involve anything from very straightforward concatenation of two different datasets, to more complicated database-style joins and merges that correctly handle any overlaps between the datasets. Series and DataFrames are built with this type of operation in mind, and Pandas includes functions and methods that make this sort of data wrangling fast and straightforward.

Here we'll take a look at simple concatenation of Series and DataFrames with the pd.concat function; later we'll dive into more sophisticated in-memory merges and joins implemented in Pandas.

pandas.concat concatenates or “stacks” together objects along an axis.
pandas.merge connects rows in DataFrames based on one or more keys.

In [2]:
import pandas as pd
import numpy as np

In [5]:
# Function to create a DataFrame:

def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}
    return pd.DataFrame(data, ind)

# Example DataFrame
make_df('ABC', range(3))


,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In addition, we'll create a quick class that allows us to display multiple DataFrames side by side. The code makes use of the special _repr_html_ method, which IPython uses to implement its rich object display:

In [6]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


#### Concatenation of Numpy Arrays

Concatenation of Series and DataFrame objects is very similar to concatenation of Numpy arrays, which can be done via the np.concatenate function as discussed in The Basics of NumPy Arrays. Recall that with it, you can combine the contents of two or more arrays into a single array:

In [7]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])


array([1, 2, 3, 4, 5, 6, 7, 8, 9])

The first argument is a list or tuple of arrays to concatenate. Additionally, it takes an axis keyword that allows you to specify the axis along which the result will be concatenated:

In [10]:
x = [[1, 2],
    [3, 4]]
np.concatenate([x, x], axis=1)


array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

#### Simple Concatenation with pd.concat ( )

Pandas has a function, pd.concat(), which has a similar syntax to np.concatenate but contains a number of options that we'll discuss momentarily:

pd.concat( ) can be used for a simple concatenation of Series or DataFrame objects, just as np.concatenate( ) can be used for simple concatenations of arrays:

In [11]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1,2,3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4,5,6])

pd.concat([ser1,ser2])


1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [12]:
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])
display('df1', 'df2', 'pd.concat([df1, df2])')


,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [17]:
s1 = pd.Series([0,1], index=['a', 'b'])

s2 = pd.Series([2,3], index=['c', 'd'])

s3 = pd.Series([4,5,6], index=['e', 'f', 'g' ])

In [18]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

#### Indexing with a DataFrame’s columns

By default, the concatenation takes place row-wise within the DataFrame (i.e., axis=0). Like np.concatenate, pd.concat allows specification of an axis along which concatenation will take place. Consider the following example:

By default concat works along axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):

In [19]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,NaN,4.0
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### Concat Function Arguments:

 objs: List or dict of pandas objects to be concatenated; this is the only required argument
 
 axis: Axis to concatenate along; defaults to 0 (along rows)
 
 join: Either 'inner' or 'outer' ('outer' by default); whether to intersection (inner) or union (outer) together indexes along the other axes
 
 join_axes: Specific indexes to use for the other n–1 axes instead of performing union/intersection logic
 
 keys: Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can either be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels)
                                   

#### Reshaping with Hierarchical Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

    stack
This “rotates” or pivots from the columns in the data to the rows

    unstack
This pivots from the rows into the columns

In [28]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns =pd.Index(['one', 'two', 'three'],
                    name='number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the stack method on this data pivots the columns into the rows, producing a Series:

In [32]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

From a hierarchically indexed Series, you can rearrange the data back into a Data‐
Frame with unstack:

In [33]:
result.unstack()


number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### Pivot "Long" to "Wide" Format

Let’s load some example data and do a small amount of time series wrangling and other data cleaning:

#### Combining and Merging Datasets

pandas.merge connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.

pandas.concat concatenates or “stacks” together objects along an axis.

#### Adding Multi-Index Keys

#### Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

#### Reordering and Sorting by Level

At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The swaplevel takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):